# Multi-Output Regression for Weather-Based Casualty Prediction

This notebook implements a complete multi-output regression pipeline that predicts multiple casualty counts from weather data using XGBoost.

## Pipeline Components:
- Data loading and exploration
- Weather feature preprocessing and engineering
- Feature scaling
- Train-test split
- Multi-output XGBoost model training
- Comprehensive evaluation metrics
- Results visualization and summary

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# XGBoost for regression
from xgboost import XGBRegressor

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")

## 2. Load and Explore Dataset

In [ ]:
# Load the dataset
try:
    df = pd.read_csv("weather_casualty.csv")
    print(f"Dataset loaded successfully! Shape: {df.shape}")
except FileNotFoundError:
    print("Error: weather_casualty.csv file not found. Please ensure the file exists in the current directory.")
    print("For demonstration purposes, creating a synthetic dataset...")
    
    # Create a synthetic weather-casualty dataset
    np.random.seed(42)
    n_samples = 1500
    
    # Weather features
    df = pd.DataFrame({
        # Temperature features
        'max_temperature': np.random.normal(32, 8, n_samples),
        'min_temperature': np.random.normal(25, 6, n_samples),
        'avg_temperature': np.random.normal(28, 7, n_samples),
        
        # Humidity features
        'max_humidity': np.random.uniform(60, 100, n_samples),
        'min_humidity': np.random.uniform(40, 80, n_samples),
        'avg_humidity': np.random.uniform(50, 90, n_samples),
        
        # Wind features
        'max_wind_speed': np.random.exponential(15, n_samples),
        'avg_wind_speed': np.random.exponential(10, n_samples),
        'wind_direction': np.random.uniform(0, 360, n_samples),
        
        # Precipitation features
        'total_rainfall': np.random.exponential(20, n_samples),
        'max_rainfall_1hr': np.random.exponential(5, n_samples),
        'rainfall_duration': np.random.uniform(0, 24, n_samples),
        
        # Pressure features
        'sea_level_pressure': np.random.normal(1013, 15, n_samples),
        'pressure_change': np.random.normal(0, 5, n_samples),
        
        # Additional weather indicators
        'visibility': np.random.uniform(1, 10, n_samples),
        'cloud_cover': np.random.uniform(0, 100, n_samples)
    })
    
    # Create casualty targets with realistic relationships to weather
    # More severe weather conditions lead to higher casualties
    severity_factor = (
        (df['max_wind_speed'] / 50) + 
        (df['total_rainfall'] / 100) + 
        (np.abs(df['pressure_change']) / 20) +
        ((100 - df['visibility']) / 100)
    )
    
    df['deaths'] = np.maximum(0, np.round(
        severity_factor * np.random.poisson(2, n_samples) + 
        np.random.normal(0, 1, n_samples)
    ).astype(int))
    
    df['injured'] = np.maximum(0, np.round(
        severity_factor * np.random.poisson(8, n_samples) + 
        np.random.normal(0, 3, n_samples)
    ).astype(int))
    
    df['missing'] = np.maximum(0, np.round(
        severity_factor * np.random.poisson(3, n_samples) + 
        np.random.normal(0, 2, n_samples)
    ).astype(int))
    
    df['displaced'] = np.maximum(0, np.round(
        severity_factor * np.random.poisson(50, n_samples) + 
        np.random.normal(0, 20, n_samples)
    ).astype(int))
    
    print(f"Synthetic dataset created! Shape: {df.shape}")

In [ ]:
# Display basic information about the dataset
print("Dataset Head:")
print(df.head())
print("\n" + "="*60 + "\n")

print("Dataset Info:")
print(df.info())
print("\n" + "="*60 + "\n")

print("Dataset Description:")
print(df.describe())
print("\n" + "="*60 + "\n")

print("Missing Values Count:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("No missing values found.")
else:
    print(f"Total missing values: {missing_values.sum()}")

## 3. Feature Engineering and Preprocessing

In [ ]:
def identify_feature_columns(df):
    """Identify weather feature columns and casualty target columns"""
    
    # Weather-related features (excluding casualty columns)
    casualty_keywords = ['death', 'injur', 'missing', 'displace', 'casualt', 'fatali']
    
    weather_features = []
    casualty_targets = []
    
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in casualty_keywords):
            casualty_targets.append(col)
        else:
            # Assume all other numeric columns are weather features
            if df[col].dtype in ['int64', 'float64']:
                weather_features.append(col)
    
    return weather_features, casualty_targets

def handle_missing_values(df, weather_features):
    """Handle missing values in weather features"""
    df_clean = df.copy()
    
    for col in weather_features:
        if df_clean[col].isnull().sum() > 0:
            # Fill missing values with median for weather data
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"Filled {df[col].isnull().sum()} missing values in {col} with median: {median_val:.2f}")
    
    return df_clean

def engineer_weather_features(df, weather_features):
    """Create engineered weather features"""
    df_engineered = df.copy()
    new_features = []
    
    # Temperature-based features
    if 'max_temperature' in weather_features and 'min_temperature' in weather_features:
        df_engineered['temperature_range'] = df_engineered['max_temperature'] - df_engineered['min_temperature']
        new_features.append('temperature_range')
    
    if 'avg_temperature' in weather_features:
        df_engineered['temp_extreme_cold'] = (df_engineered['avg_temperature'] < 20).astype(int)
        df_engineered['temp_extreme_hot'] = (df_engineered['avg_temperature'] > 35).astype(int)
        new_features.extend(['temp_extreme_cold', 'temp_extreme_hot'])
    
    # Humidity-based features
    if 'max_humidity' in weather_features and 'min_humidity' in weather_features:
        df_engineered['humidity_range'] = df_engineered['max_humidity'] - df_engineered['min_humidity']
        new_features.append('humidity_range')
    
    if 'avg_humidity' in weather_features:
        df_engineered['high_humidity'] = (df_engineered['avg_humidity'] > 80).astype(int)
        new_features.append('high_humidity')
    
    # Wind-based features
    if 'max_wind_speed' in weather_features:
        df_engineered['wind_category'] = pd.cut(df_engineered['max_wind_speed'], 
                                               bins=[0, 10, 25, 50, np.inf], 
                                               labels=['Light', 'Moderate', 'Strong', 'Severe']).astype('category').cat.codes
        df_engineered['extreme_wind'] = (df_engineered['max_wind_speed'] > 40).astype(int)
        new_features.extend(['wind_category', 'extreme_wind'])
    
    # Rainfall-based features
    if 'total_rainfall' in weather_features:
        df_engineered['rainfall_category'] = pd.cut(df_engineered['total_rainfall'], 
                                                  bins=[0, 5, 20, 50, np.inf], 
                                                  labels=['Light', 'Moderate', 'Heavy', 'Extreme']).astype('category').cat.codes
        df_engineered['heavy_rain'] = (df_engineered['total_rainfall'] > 30).astype(int)
        df_engineered['no_rain'] = (df_engineered['total_rainfall'] == 0).astype(int)
        new_features.extend(['rainfall_category', 'heavy_rain', 'no_rain'])
    
    # Pressure-based features
    if 'pressure_change' in weather_features:
        df_engineered['rapid_pressure_drop'] = (df_engineered['pressure_change'] < -5).astype(int)
        df_engineered['rapid_pressure_rise'] = (df_engineered['pressure_change'] > 5).astype(int)
        new_features.extend(['rapid_pressure_drop', 'rapid_pressure_rise'])
    
    # Visibility features
    if 'visibility' in weather_features:
        df_engineered['poor_visibility'] = (df_engineered['visibility'] < 3).astype(int)
        new_features.append('poor_visibility')
    
    # Interaction terms
    if 'max_wind_speed' in weather_features and 'total_rainfall' in weather_features:
        df_engineered['wind_rain_interaction'] = df_engineered['max_wind_speed'] * df_engineered['total_rainfall']
        new_features.append('wind_rain_interaction')
    
    if 'avg_temperature' in weather_features and 'avg_humidity' in weather_features:
        df_engineered['heat_humidity_index'] = df_engineered['avg_temperature'] * df_engineered['avg_humidity'] / 100
        new_features.append('heat_humidity_index')
    
    print(f"Engineered {len(new_features)} new features: {new_features}")
    
    return df_engineered, new_features

# Apply preprocessing
print("Identifying feature and target columns...")
weather_features, casualty_targets = identify_feature_columns(df)

print(f"Weather features ({len(weather_features)}): {weather_features}")
print(f"Casualty targets ({len(casualty_targets)}): {casualty_targets}")

# Handle missing values
print("\nHandling missing values...")
df_clean = handle_missing_values(df, weather_features)

# Engineer features
print("\nEngineering weather features...")
df_processed, new_features = engineer_weather_features(df_clean, weather_features)

# Update feature list
all_weather_features = weather_features + new_features
print(f"\nTotal weather features: {len(all_weather_features)}")

## 4. Feature-Target Separation and Analysis

In [ ]:
# Separate features and targets
X = df_processed[all_weather_features]
y = df_processed[casualty_targets]

print(f"Feature matrix shape: {X.shape}")
print(f"Target matrix shape: {y.shape}")

# Display target statistics
print("\nTarget Variable Statistics:")
print(y.describe())

# Check for any remaining missing values
print("\nMissing values check:")
print(f"Features missing values: {X.isnull().sum().sum()}")
print(f"Targets missing values: {y.isnull().sum().sum()}")

# Display correlation between targets
print("\nCorrelation between target variables:")
target_corr = y.corr()
print(target_corr.round(3))

## 5. Train-Test Split

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Training set - Features: {X_train.shape}, Targets: {y_train.shape}")
print(f"Testing set - Features: {X_test.shape}, Targets: {y_test.shape}")
print(f"Train-test split ratio: {len(X_train)}/{len(X_test)} = {len(X_train)/len(X_test):.2f}")

# Display training set target statistics
print("\nTraining set target statistics:")
print(y_train.describe().round(2))

## 6. Feature Scaling

In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on training data only
print("Fitting scaler on training weather features...")
X_train_scaled = scaler.fit_transform(X_train)

# Transform both training and testing data
print("Scaling training and testing weather features...")
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print(f"Scaled training features shape: {X_train_scaled.shape}")
print(f"Scaled testing features shape: {X_test_scaled.shape}")

# Show scaling statistics
print("\nScaling validation (training data):")
print(f"Original features - Mean: {X_train.mean().mean():.4f}, Std: {X_train.std().mean():.4f}")
print(f"Scaled features - Mean: {X_train_scaled.mean().mean():.4f}, Std: {X_train_scaled.std().mean():.4f}")

## 7. Multi-Output XGBoost Model Training

In [ ]:
# Configure XGBoost base estimator
xgb_estimator = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,  # Use all available cores
    verbosity=0  # Suppress XGBoost output
)

# Create multi-output regressor
print("Initializing Multi-Output XGBoost Regressor...")
multi_output_model = MultiOutputRegressor(xgb_estimator)

# Train the model
print("Training multi-output model...")
print("This may take a few minutes depending on dataset size...")

multi_output_model.fit(X_train_scaled, y_train)

print("Model training completed successfully!")

# Display model information
print(f"\nModel Information:")
print(f"Number of outputs: {len(multi_output_model.estimators_)}")
print(f"Output targets: {list(y.columns)}")
print(f"Number of features: {X_train_scaled.shape[1]}")

## optuna hyperparmas

## 8. Model Prediction and Evaluation

In [ ]:
# Make predictions
print("Generating predictions...")
y_train_pred = multi_output_model.predict(X_train_scaled)
y_test_pred = multi_output_model.predict(X_test_scaled)

# Convert predictions to DataFrames
y_train_pred_df = pd.DataFrame(y_train_pred, columns=y.columns, index=y_train.index)
y_test_pred_df = pd.DataFrame(y_test_pred, columns=y.columns, index=y_test.index)

print("Predictions generated successfully!")
print(f"Training predictions shape: {y_train_pred_df.shape}")
print(f"Testing predictions shape: {y_test_pred_df.shape}")

In [ ]:
def calculate_multi_output_metrics(y_true, y_pred, dataset_name="Dataset"):
    """Calculate comprehensive metrics for multi-output regression"""
    
    results = []
    
    for i, col in enumerate(y_true.columns):
        # Extract true and predicted values for this output
        y_true_col = y_true.iloc[:, i] if isinstance(y_true, pd.DataFrame) else y_true[:, i]
        y_pred_col = y_pred.iloc[:, i] if isinstance(y_pred, pd.DataFrame) else y_pred[:, i]
        
        # Calculate metrics
        mae = mean_absolute_error(y_true_col, y_pred_col)
        mse = mean_squared_error(y_true_col, y_pred_col)
        rmse = np.sqrt(mse)
        
        results.append({
            'Dataset': dataset_name,
            'Output': col,
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse
        })
    
    return pd.DataFrame(results)

# Calculate metrics for training and testing sets
print("Calculating evaluation metrics...")

train_metrics = calculate_multi_output_metrics(y_train, y_train_pred_df, "Training")
test_metrics = calculate_multi_output_metrics(y_test, y_test_pred_df, "Testing")

# Combine results
all_metrics = pd.concat([train_metrics, test_metrics], ignore_index=True)

print("\n" + "="*80)
print("MULTI-OUTPUT REGRESSION EVALUATION RESULTS")
print("="*80)
print(all_metrics.round(4))

In [ ]:
# Create summary results table
print("\n" + "="*60)
print("SUMMARY RESULTS TABLE")
print("="*60)

# Create pivot table for better visualization
summary_table = test_metrics[['Output', 'MAE', 'RMSE']].copy()
summary_table = summary_table.round(4)

print("Test Set Performance by Output:")
print(summary_table.to_string(index=False))

# Calculate overall averages
avg_mae = test_metrics['MAE'].mean()
avg_rmse = test_metrics['RMSE'].mean()

print(f"\nOverall Averages (Test Set):")
print(f"Average MAE: {avg_mae:.4f}")
print(f"Average RMSE: {avg_rmse:.4f}")

# Performance assessment
print(f"\nModel Performance Assessment:")
for idx, row in test_metrics.iterrows():
    output = row['Output']
    rmse = row['RMSE']
    mae = row['MAE']
    
    # Get target statistics for context
    target_mean = y_test[output].mean()
    target_std = y_test[output].std()
    
    mae_ratio = mae / target_mean if target_mean > 0 else float('inf')
    rmse_ratio = rmse / target_std if target_std > 0 else float('inf')
    
    print(f"- {output}: MAE/Mean = {mae_ratio:.3f}, RMSE/Std = {rmse_ratio:.3f}")

## 10. Results Visualization

In [ ]:
# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Create visualization plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Multi-Output Regression Model Performance', fontsize=16, fontweight='bold')

# Plot 1: Actual vs Predicted for each target
colors = ['blue', 'red', 'green', 'orange']
for i, target in enumerate(casualty_targets):
    if i < 4:  # Limit to 4 subplots
        row, col = i // 2, i % 2
        
        y_true_vals = y_test[target].values
        y_pred_vals = y_test_pred_df[target].values
        
        axes[row, col].scatter(y_true_vals, y_pred_vals, alpha=0.6, color=colors[i])
        
        # Add perfect prediction line
        min_val = min(y_true_vals.min(), y_pred_vals.min())
        max_val = max(y_true_vals.max(), y_pred_vals.max())
        axes[row, col].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
        
        axes[row, col].set_xlabel(f'Actual {target}')
        axes[row, col].set_ylabel(f'Predicted {target}')
        axes[row, col].set_title(f'{target} - Actual vs Predicted')
        axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot 2: Feature Importance Heatmap
plt.figure(figsize=(12, 8))

# Create feature importance matrix
importance_matrix = feature_importance_df.pivot_table(
    values='Importance', 
    index='Feature', 
    columns='Target', 
    fill_value=0
)

# Show only top 20 features
top_20_features = avg_importance.head(20).index
importance_matrix_top = importance_matrix.loc[top_20_features]

sns.heatmap(importance_matrix_top, 
            annot=True, 
            fmt='.3f', 
            cmap='YlOrRd', 
            cbar_kws={'label': 'Feature Importance'})

plt.title('Feature Importance Heatmap (Top 20 Features)', fontsize=14, fontweight='bold')
plt.xlabel('Casualty Targets')
plt.ylabel('Weather Features')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 11. Model Summary and Output

In [ ]:
# Create comprehensive model summary
print("\n" + "="*80)
print("MULTI-OUTPUT REGRESSION MODEL SUMMARY")
print("="*80)

print(f"Dataset Information:")
print(f"- Total samples: {len(df_processed)}")
print(f"- Weather features (original): {len(weather_features)}")
print(f"- Weather features (engineered): {len(all_weather_features)}")
print(f"- Casualty targets: {len(casualty_targets)}")
print(f"- Training samples: {len(X_train)}")
print(f"- Testing samples: {len(X_test)}")

print(f"\nModel Configuration:")
print(f"- Algorithm: Multi-Output XGBoost Regression")
print(f"- Base estimators: {len(multi_output_model.estimators_)}")
print(f"- XGBoost parameters:")
print(f"  * n_estimators: 300")
print(f"  * learning_rate: 0.05")
print(f"  * max_depth: 5")
print(f"  * subsample: 0.8")
print(f"  * colsample_bytree: 0.8")

print(f"\nModel Performance (Test Set):")
for idx, row in test_metrics.iterrows():
    print(f"- {row['Output']:<12}: MAE={row['MAE']:<8.4f} RMSE={row['RMSE']:<8.4f}")

print(f"\nOverall Performance:")
print(f"- Average MAE: {avg_mae:.4f}")
print(f"- Average RMSE: {avg_rmse:.4f}")

# Check for overfitting
train_avg_rmse = train_metrics['RMSE'].mean()
test_avg_rmse = test_metrics['RMSE'].mean()
overfitting_ratio = test_avg_rmse / train_avg_rmse

print(f"\nGeneralization Assessment:")
print(f"- Training RMSE: {train_avg_rmse:.4f}")
print(f"- Testing RMSE: {test_avg_rmse:.4f}")
print(f"- Generalization ratio: {overfitting_ratio:.3f}")

if overfitting_ratio > 1.2:
    print("- Warning: Model may be overfitting (test RMSE >> train RMSE)")
elif overfitting_ratio < 0.9:
    print("- Note: Test performance better than training (possible data leakage?)")
else:
    print("- Good: Model generalizes well to unseen data")

In [ ]:
# Save model outputs
import joblib
from datetime import datetime

# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# print("Saving model outputs...")

# # Save the trained model
# model_filename = f'multi_output_xgb_model_{timestamp}.pkl'
# with open(model_filename, 'wb') as f:
#     pickle.dump(multi_output_model, f)

# # Save the scaler
# scaler_filename = f'weather_scaler_{timestamp}.pkl'
# with open(scaler_filename, 'wb') as f:
#     pickle.dump(scaler, f)

# # Save predictions
# predictions_filename = f'casualty_predictions_{timestamp}.csv'
# predictions_output = pd.concat([
#     y_test.reset_index(drop=True),
#     y_test_pred_df.reset_index(drop=True).add_suffix('_predicted')
# ], axis=1)
# predictions_output.to_csv(predictions_filename, index=False)

# # Save evaluation metrics
# metrics_filename = f'model_evaluation_metrics_{timestamp}.csv'
# all_metrics.to_csv(metrics_filename, index=False)

# # Save feature importance
# importance_filename = f'feature_importance_{timestamp}.csv'
# feature_importance_df.to_csv(importance_filename, index=False)

# print(f"\nFiles saved:")
# print(f"- {model_filename} (trained multi-output model)")
# print(f"- {scaler_filename} (fitted feature scaler)")
# print(f"- {predictions_filename} (test predictions vs actual)")
# print(f"- {metrics_filename} (evaluation metrics)")
# print(f"- {importance_filename} (feature importance scores)")

# print(f"\n" + "="*80)
# print("MULTI-OUTPUT REGRESSION PIPELINE COMPLETED SUCCESSFULLY!")
# print("="*80)
# print("✓ Weather data preprocessing and feature engineering completed")
# print("✓ Features scaled and train-test split performed")
# print("✓ Multi-output XGBoost model trained successfully")
# print("✓ Comprehensive evaluation metrics calculated")
# print("✓ Feature importance analysis completed")
# print("✓ Results visualized and saved")
# print(f"✓ Average model RMSE: {avg_rmse:.4f}")